In [9]:
import pandas as pd
import os
from pathlib import Path
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from ultralytics import YOLO
import cv2
import collections

reformat the dataset labels for YOLO format

In [22]:
TRAIN_CSV_PATH = Path('../tracknet/data/preprocessed_tracknet_dataset/labels_train.csv') 
VAL_CSV_PATH = Path('../tracknet/data/preprocessed_tracknet_dataset/labels_val.csv') 
RAW_IMAGE_DIR = Path('../tracknet/data/preprocessed_tracknet_dataset/images')
YOLO_DATASET_DIR = Path('data/yolo_tennis_ball')

IMG_WIDTH = 1280
IMG_HEIGHT = 720
BOX_WIDTH = 20
BOX_HEIGHT = 20

# --- Create YOLO directories ---
os.makedirs(YOLO_DATASET_DIR / 'images/train', exist_ok=True)
os.makedirs(YOLO_DATASET_DIR / 'images/val', exist_ok=True)
os.makedirs(YOLO_DATASET_DIR / 'labels/train', exist_ok=True)
os.makedirs(YOLO_DATASET_DIR / 'labels/val', exist_ok=True)

# --- Load and filter data ---
df_train_og = pd.read_csv(TRAIN_CSV_PATH)
df_val_og = pd.read_csv(VAL_CSV_PATH)

full_df = pd.concat([df_train_og, df_val_og], ignore_index=True)

#use only frames where a ball is clearly visible
df_filtered = full_df[full_df['visibility'].isin([1,2,3])].copy()

# --- Split data ---
train_df, val_df = train_test_split(df_filtered, test_size=0.2, random_state=42)
print(f"Training Samples: {len(train_df)}, Validation Samples: {len(val_df)}")

# --- Process and save data ---
def process_split(df, split_name):
    print(f"Processing {split_name} split...")
    for _, row in tqdm(df.iterrows(), total=len(df)):
        img_path_rel = row['path1']
        src_img_path = RAW_IMAGE_DIR / img_path_rel.replace('images/', '')
        
        # Create a clean filename for the new location
        new_filename_base = src_img_path.stem
        dest_img_path = YOLO_DATASET_DIR / f'images/{split_name}/{new_filename_base}.jpg'
        
        # Copy the image file
        os.system(f'cp "{src_img_path}" "{dest_img_path}"')
        
        # Create the label file
        x, y = row['x-coordinate'], row['y-coordinate']
        
        # Normalize coordinates
        x_center_norm = x / IMG_WIDTH
        y_center_norm = y / IMG_HEIGHT
        width_norm = BOX_WIDTH / IMG_WIDTH
        height_norm = BOX_HEIGHT / IMG_HEIGHT
        
        # Write to .txt file
        label_path = YOLO_DATASET_DIR / f'labels/{split_name}/{new_filename_base}.txt'
        with open(label_path, 'w') as f:
            f.write(f"0 {x_center_norm} {y_center_norm} {width_norm} {height_norm}\n")

process_split(train_df, 'train')
process_split(val_df, 'val')

print("YOLO dataset preparation complete!")

Training Samples: 15140, Validation Samples: 3786
Processing train split...


100%|██████████| 15140/15140 [08:18<00:00, 30.36it/s]


Processing val split...


100%|██████████| 3786/3786 [02:03<00:00, 30.75it/s]

YOLO dataset preparation complete!


train the YOLOv8 Model

In [ ]:
# Load a pretrained model 
model = YOLO('yolov8m.pt')

#Train the model
# the ultralytics API handles the entire training loop 
results = model.train(
    data=str(YOLO_DATASET_DIR / 'dataset.yaml'),
    epochs=150,       # Start with fewer epochs
    imgsz=640,       # Train on smaller images for speed
    batch=16,
    name='yolov8m_tennis_ballv2_150epochs' # Name for the experiment run
)


New https://pypi.org/project/ultralytics/8.3.220 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.218  Python-3.13.7 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data\yolo_tennis_ball\dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8m.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8m_ten

In [ ]:
model = YOLO('runs/detect/yolov8m_tennis_ballv2_150epochs/weights/best.pt')

# Run the validation
# This will use the validation data specified in your dataset.yaml file
metrics = model.val()

Ultralytics 8.3.218  Python-3.13.7 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 12282MiB)
Model summary (fused): 92 layers, 25,840,339 parameters, 0 gradients, 78.7 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 1647.5436.8 MB/s, size: 142.4 KB)
val: Scanning C:\Users\jake\dev\IP_A2\TracknetImplementation\Tennis-Ball-Tracking\src\trackers\yolo\data\yolo_tennis_ball\labels\val.cache... 620 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 620/620 804.1Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 39/39 10.8it/s 3.6s0.1s
                   all        620        620      0.878      0.732      0.818      0.413
Speed: 0.4ms preprocess, 3.6ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to C:\Users\jake\dev\IP_A2\TracknetImplementation\Tennis-Ball-Tracking\src\trackers\yolo\runs\detect\val
